## KAGGLE TITANIC



### 1. Data Cleaning

In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [12]:
data_train=pd.read_csv("data/train.csv")
data_train=shuffle(data_train,random_state=3)

In [13]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
395,396,0,3,"Johansson, Mr. Erik",male,22.0,0,0,350052,7.7958,NaN,S
85,86,1,3,"Backstrom, Mrs. Karl Alfred (Maria Mathilda Gu...",female,33.0,3,0,3101278,15.8500,NaN,S
201,202,0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.5500,NaN,S
542,543,0,3,"Andersson, Miss. Sigrid Elisabeth",female,11.0,4,2,347082,31.2750,NaN,S
702,703,0,3,"Barbara, Miss. Saiide",female,18.0,0,1,2691,14.4542,NaN,C


In [14]:
cabin=data_train[['Cabin']]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder=OrdinalEncoder()
cabin_encoded=ordinal_encoder.fit_transform(cabin)
ordinal_encoder.categories_

[array(['A10', 'A14', 'A16', 'A19', 'A20', 'A23', 'A24', 'A26', 'A31',
        'A32', 'A34', 'A36', 'A5', 'A6', 'A7', 'B101', 'B102', 'B18',
        'B19', 'B20', 'B22', 'B28', 'B3', 'B30', 'B35', 'B37', 'B38',
        'B39', 'B4', 'B41', 'B42', 'B49', 'B5', 'B50', 'B51 B53 B55',
        'B57 B59 B63 B66', 'B58 B60', 'B69', 'B71', 'B73', 'B77', 'B78',
        'B79', 'B80', 'B82 B84', 'B86', 'B94', 'B96 B98', 'C101', 'C103',
        'C104', 'C106', 'C110', 'C111', 'C118', 'C123', 'C124', 'C125',
        'C126', 'C128', 'C148', 'C2', 'C22 C26', 'C23 C25 C27', 'C30',
        'C32', 'C45', 'C46', 'C47', 'C49', 'C50', 'C52', 'C54', 'C62 C64',
        'C65', 'C68', 'C7', 'C70', 'C78', 'C82', 'C83', 'C85', 'C86',
        'C87', 'C90', 'C91', 'C92', 'C93', 'C95', 'C99', 'D', 'D10 D12',
        'D11', 'D15', 'D17', 'D19', 'D20', 'D21', 'D26', 'D28', 'D30',
        'D33', 'D35', 'D36', 'D37', 'D45', 'D46', 'D47', 'D48', 'D49',
        'D50', 'D56', 'D6', 'D7', 'D9', 'E10', 'E101', 'E12', 'E121',

In [15]:
data_train[['Cabin']]=cabin_encoded

In [16]:
train_labels=data_train["Survived"]
data_train.loc[data_train['Sex'] =='male','s'] = 1 
data_train.loc[data_train['Sex'] =='female','s'] = 0
data_train=data_train.drop(['PassengerId','Name','Ticket','Embarked','Sex','Survived'],axis=1)
data_train.head()

,Pclass,Age,SibSp,Parch,Fare,Cabin,s
395,3,22.0,0,0,7.7958,NaN,1.0
85,3,33.0,3,0,15.8500,NaN,0.0
201,3,NaN,8,2,69.5500,NaN,1.0
542,3,11.0,4,2,31.2750,NaN,0.0
702,3,18.0,0,1,14.4542,NaN,0.0


In [17]:
data_train.to_numpy()
data_train=np.nan_to_num(data_train)
train_labels.to_numpy()
train_examples=data_train

In [18]:
X = train_examples
y = train_labels

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)
BATCH_SIZE = int((X_train.shape[0])/1000)

In [19]:
X_train.shape

(712, 7)

###  2. TWEAK THE MODEL

In [43]:
def get_basic_model():

  model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(7,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(14,activation='relu'),
    tf.keras.layers.Dense(7,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
    ])
  opt = tf.keras.optimizers.SGD(learning_rate=0.01)
  model.compile(optimizer=opt,
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [44]:
ACCURACY_THRESHOLD = 0.9
# Implement callback function to stop training
# when accuracy reaches e.g. ACCURACY_THRESHOLD = 0.95
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True
callbacks = myCallback()


In [45]:
model = get_basic_model()
model.fit(X_train,y_train, epochs=100, batch_size=BATCH_SIZE,validation_data=(X_valid,y_valid),callbacks=[callbacks])

Epoch 1/100
 1/23 [>.............................] - ETA: 5s - loss: 0.8416 - accuracy: 0.3750

2022-02-06 15:45:23.123600: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 19ms/step - loss: 0.7984 - accuracy: 0.4157 - val_loss: 0.9553 - val_accuracy: 0.3258
Epoch 2/100
16/23 [===================>..........] - ETA: 0s - loss: 0.7754 - accuracy: 0.4395

2022-02-06 15:45:23.617019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 0s 7ms/step - loss: 0.7898 - accuracy: 0.4480 - val_loss: 0.8471 - val_accuracy: 0.4607
Epoch 3/100
23/23 [==============================] - 0s 8ms/step - loss: 0.7790 - accuracy: 0.4565 - val_loss: 0.8032 - val_accuracy: 0.5618
Epoch 4/100
23/23 [==============================] - 0s 7ms/step - loss: 0.7706 - accuracy: 0.4888 - val_loss: 0.7799 - val_accuracy: 0.5955
Epoch 5/100
23/23 [==============================] - 0s 6ms/step - loss: 0.7549 - accuracy: 0.5154 - val_loss: 0.7650 - val_accuracy: 0.6067
Epoch 6/100
23/23 [==============================] - 0s 6ms/step - loss: 0.7432 - accuracy: 0.5463 - val_loss: 0.7545 - val_accuracy: 0.6404
Epoch 7/100
23/23 [==============================] - 0s 6ms/step - loss: 0.7338 - accuracy: 0.5520 - val_loss: 0.7448 - val_accuracy: 0.6180
Epoch 8/100
23/23 [==============================] - 0s 6ms/step - loss: 0.7281 - accuracy: 0.5730 - val_loss: 0.7375 - val_accuracy: 0.6067
Epoch 9/100
23/23 [======

In [46]:
model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 12ms/step - loss: 0.4628 - accuracy: 0.8000


[0.46284323930740356, 0.800000011920929]

### 3. CSV SUBMISSION

In [36]:
test=pd.read_csv("data/test.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [40]:
test=pd.read_csv("data/test.csv")
test_id=test["PassengerId"]
test.loc[test['Sex'] =='male','s'] = 1 
test.loc[test['Sex'] =='female','s'] = 0
cabin=test[['Cabin']]
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder=OrdinalEncoder()
cabin_encoded=ordinal_encoder.fit_transform(cabin)
test[['Cabin']]=cabin_encoded
test=test.drop(['Name','Ticket','Embarked','Sex','PassengerId'],axis=1)
test.head()


,Pclass,Age,SibSp,Parch,Fare,Cabin,s
0,3,34.5,0,0,7.8292,NaN,1.0
1,3,47.0,1,0,7.0000,NaN,0.0
2,2,62.0,0,0,9.6875,NaN,1.0
3,3,27.0,0,0,8.6625,NaN,1.0
4,3,22.0,1,1,12.2875,NaN,0.0


In [41]:
test.to_numpy()
test=np.nan_to_num(test)
ans=model.predict(test)
data=pd.DataFrame()
ans=np.where(ans > 0.5, 1, 0)
data.insert(0,"PassengerId",test_id)
data.insert(1,"Survived",ans)


In [42]:
data.to_csv("data/model1_17.csv",index=False)